# Texto Bruto (Com Header e Com Entrevistador)

In [15]:
# Carregar X_train e y_train
import os
import chardet

def load_texts_from_directory(directory, label):
    texts = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".cha"):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'rb') as file:
                raw_data = file.read()
                result = chardet.detect(raw_data)
                text = raw_data.decode(result['encoding'])
                texts.append(text)
                labels.append(label)
    return texts, labels

# Diretórios das pastas
negative_dir = r'C:\Users\Lenovo\Desktop\IC\[98] ADRESS-20 Database\ADReSS-IS2020-data\train\transcription\cc'
positive_dir = r'C:\Users\Lenovo\Desktop\IC\[98] ADRESS-20 Database\ADReSS-IS2020-data\train\transcription\cd'

# Carregar textos negativos (rótulo 0)
x_neg, y_neg = load_texts_from_directory(negative_dir, 0)

# Carregar textos positivos (rótulo 1)
x_pos, y_pos = load_texts_from_directory(positive_dir, 1)

# Combinar textos e rótulos
x_train = x_neg + x_pos
y_train = y_neg + y_pos

print(x_train[0])

@UTF8
@PID:	11312/t-00002184-1
@Begin
@Languages:	eng
@Participants:	PAR Participant, INV Investigator
@ID:	eng|Pitt|PAR|74;|male|Control||Participant|||
@ID:	eng|Pitt|INV|||||Investigator|||
@Media:	S001, audio
*INV:	tell me everything that you see going on in that picture . 0_2360
%mor:	v|tell pro:obj|me pro:indef|everything pro:rel|that pro:per|you
	v|see n:gerund|go-PRESP adv|on prep|in det:dem|that n|picture .
%gra:	1|0|ROOT 2|1|OBJ2 3|1|OBJ 4|6|LINK 5|6|SUBJ 6|3|CMOD 7|6|OBJ 8|6|JCT
	9|6|JCT 10|11|DET 11|9|POBJ 12|1|PUNCT
*INV:	everything that you see happening . 2360_4266
%mor:	pro:indef|everything pro:rel|that pro:per|you v|see
	n:gerund|happen-PRESP .
%gra:	1|0|INCROOT 2|4|LINK 3|4|SUBJ 4|1|CMOD 5|4|OBJ 6|1|PUNCT
*PAR:	well there's a mother standing there &uh &uh washing the dishes
	an(d) the sink is overspilling [: overflowing] [* s:r] . 4266_13310
%mor:	co|well pro:exist|there~cop|be&3S det:art|a n|mother
	part|stand-PRESP adv|there part|wash-PRESP det:art|the n|dish-P

In [16]:
# Carregar X_test
def load_texts_from_directory(directory):
    texts = []
    for filename in os.listdir(directory):
        if filename.endswith(".cha"):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'rb') as file:
                raw_data = file.read()
                result = chardet.detect(raw_data)
                text = raw_data.decode(result['encoding'])
                texts.append(text)
    return texts

# Diretório da pasta de teste
test_dir = r'C:\Users\Lenovo\Desktop\IC\[98] ADRESS-20 Database\ADReSS-IS2020-data\test\transcription'

# Carregar textos de teste
x_test = load_texts_from_directory(test_dir)

y_test = [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0]

In [17]:
# Implementar modelo de compressão
import gzip
from sklearn.neighbors import KNeighborsClassifier

def ncd(x, x2): # NCD with compressed lengths
    x_compressed = len(gzip.compress(x.encode()))
    x2_compressed = len(gzip.compress(x2.encode()))  
    xx2 = len(gzip.compress((" ".join([x,x2])).encode()))
    return (xx2 - min(x_compressed, x2_compressed)) / max(x_compressed, x2_compressed)

train_ncd = [[ncd(x_train[i], x_train[j]) for j in range(len(x_train))] for i in range(len(x_train))]
test_ncd = [[ncd(x_test[i], x_train[j]) for j in range(len(x_train))] for i in range(len(x_test))]

# KNN classification
neigh = KNeighborsClassifier(n_neighbors=7) 
neigh.fit(train_ncd, y_train)

print("Accuracy:", neigh.score(test_ncd, y_test))

Accuracy: 0.7708333333333334


## Multiprocessamento

In [18]:
import multiprocessing

# Number of processes to use
NUM_PROCESSES = 8

# NCD with compressed lengths
def ncd(x, x2):
  x_compressed = len(gzip.compress(x.encode()))
  x2_compressed = len(gzip.compress(x2.encode()))  
  xx2 = len(gzip.compress((" ".join([x,x2])).encode()))
  return (xx2 - min(x_compressed, x2_compressed)) / max(x_compressed, x2_compressed)

# Initialize NCD matrices  
train_ncd = [[0] * len(x_train) for _ in range(len(x_train))]
test_ncd = [[0] * len(x_train) for _ in range(len(x_test))]

# Helper function to compute NCD row 
def calculate_ncd_row(data_row):
    i = data_row[0]
    row = [ncd(data_row[1], x_train[j]) for j in range(len(x_train))] 
    return i, row

with multiprocessing.Pool(NUM_PROCESSES) as pool:
    # Compute train NCD
    train_data = enumerate(x_train)
    train_results = pool.map(calculate_ncd_row, train_data)
    
    # Compute test NCD
    test_data = enumerate(x_test)  
    test_results = pool.map(calculate_ncd_row, test_data)
    
# Insert rows into NCD matrices    
for i, row in train_results:
    train_ncd[i] = row
    
for i, row in test_results:
    test_ncd[i] = row

# KNN classification
neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(train_ncd, y_train)

accuracy = neigh.score(test_ncd, y_test)
print("Accuracy:", accuracy)

# Texto Com Header e Sem Entrevistador

In [13]:
# Carregar X_train e y_train
import os
import chardet
import re

def load_texts_from_directory(directory, label):
    texts = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".cha"):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'rb') as file:
                raw_data = file.read()
                result = chardet.detect(raw_data)
                text = raw_data.decode(result['encoding'])

                # Remover texto entre *INV: e %
                text = re.sub(r'\*INV:.*?%', '%', text, flags=re.DOTALL)

                texts.append(text)
                labels.append(label)
    return texts, labels

# Diretórios das pastas
negative_dir = r'C:\Users\Lenovo\Desktop\IC\[98] ADRESS-20 Database\ADReSS-IS2020-data\train\transcription\cc'
positive_dir = r'C:\Users\Lenovo\Desktop\IC\[98] ADRESS-20 Database\ADReSS-IS2020-data\train\transcription\cd'

# Carregar textos negativos (rótulo 0)
x_neg, y_neg = load_texts_from_directory(negative_dir, 0)

# Carregar textos positivos (rótulo 1)
x_pos, y_pos = load_texts_from_directory(positive_dir, 1)

# Combinar textos e rótulos
x_train = x_neg + x_pos
y_train = y_neg + y_pos

print(x_train[0])

@UTF8
@PID:	11312/t-00002184-1
@Begin
@Languages:	eng
@Participants:	PAR Participant, INV Investigator
@ID:	eng|Pitt|PAR|74;|male|Control||Participant|||
@ID:	eng|Pitt|INV|||||Investigator|||
@Media:	S001, audio
%mor:	v|tell pro:obj|me pro:indef|everything pro:rel|that pro:per|you
	v|see n:gerund|go-PRESP adv|on prep|in det:dem|that n|picture .
%gra:	1|0|ROOT 2|1|OBJ2 3|1|OBJ 4|6|LINK 5|6|SUBJ 6|3|CMOD 7|6|OBJ 8|6|JCT
	9|6|JCT 10|11|DET 11|9|POBJ 12|1|PUNCT
%mor:	pro:indef|everything pro:rel|that pro:per|you v|see
	n:gerund|happen-PRESP .
%gra:	1|0|INCROOT 2|4|LINK 3|4|SUBJ 4|1|CMOD 5|4|OBJ 6|1|PUNCT
*PAR:	well there's a mother standing there &uh &uh washing the dishes
	an(d) the sink is overspilling [: overflowing] [* s:r] . 4266_13310
%mor:	co|well pro:exist|there~cop|be&3S det:art|a n|mother
	part|stand-PRESP adv|there part|wash-PRESP det:art|the n|dish-PL
	coord|and det:art|the n|sink aux|be&3S over#part|flow-PRESP .
%gra:	1|3|COM 2|3|SUBJ 3|0|ROOT 4|5|DET 5|3|PRED 6|5|XMOD 7|6|J

In [11]:
# Carregar X_test
import os
import chardet
import re

# Função para carregar textos e remover texto entre *INV: e %
def load_texts_from_directory(directory):
    texts = []
    for filename in os.listdir(directory):
        if filename.endswith(".cha"):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'rb') as file:
                raw_data = file.read()
                result = chardet.detect(raw_data)
                text = raw_data.decode(result['encoding'])

                # Remover texto entre *INV: e %
                text = re.sub(r'\*INV:.*?%', '%', text, flags=re.DOTALL)

                texts.append(text)
    return texts

# Diretório da pasta de teste
test_dir = r'C:\Users\Lenovo\Desktop\IC\[98] ADRESS-20 Database\ADReSS-IS2020-data\test\transcription'

# Carregar textos de teste
x_test = load_texts_from_directory(test_dir)

y_test = [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0]

In [12]:
# Implementar modelo de compressão
import gzip
from sklearn.neighbors import KNeighborsClassifier

def ncd(x, x2): # NCD with compressed lengths
    x_compressed = len(gzip.compress(x.encode()))
    x2_compressed = len(gzip.compress(x2.encode()))  
    xx2 = len(gzip.compress((" ".join([x,x2])).encode()))
    return (xx2 - min(x_compressed, x2_compressed)) / max(x_compressed, x2_compressed)

train_ncd = [[ncd(x_train[i], x_train[j]) for j in range(len(x_train))] for i in range(len(x_train))]
test_ncd = [[ncd(x_test[i], x_train[j]) for j in range(len(x_train))] for i in range(len(x_test))]

# KNN classification
neigh = KNeighborsClassifier(n_neighbors=7) 
neigh.fit(train_ncd, y_train)

print("Accuracy:", neigh.score(test_ncd, y_test))

Accuracy: 0.75
